In [12]:
%%file customConv3d.py
#Подключение модулей
import numpy as np
import torch

def customConv3d(in_channels, out_channels, kernel_size, stride=1, padding=0\
    , dilation=1, groups=1, bias=True, padding_mode="zeros"):
    #Обёртка
    def wrapper(tens):
        #Проверки параметров
        if (in_channels%groups !=0) or (out_channels%groups !=0):
            raise Exception(f"%s must be divisible by groups" % ("in_channels" if in_channels%groups !=0 else "out_channels"))
        if padding < 0:
            raise Exception(f"Padiing should be 0 or positive")
        if stride < 0:
            raise Exception(f"Stride should be 0 or positive")
        if groups < 0:
            raise Exception(f"Groups should be positive")  
        
        #Смещение
        if bias:
            bias_value = torch.rand(out_channels)
        else:
            bias_value = torch.zeros(out_channels)
        
        #Подложка
        if (padding_mode == 'zeros'):
            pad = torch.nn.ZeroPad2d(padding)
            tens = pad(tens)
        elif (padding_mode == 'reflect'):
            pad = torch.nn.ReflectionPad2d(padding)
            tens = pad(tens)
        elif (padding_mode == 'replicate'):
            pad = torch.nn.ReplicationPad2d(padding)
            tens = pad(tens)
        elif (padding_mode == 'circular'):
            pad = torch.nn.CircularPad2d(padding)
            tens = pad(tens)
        else:
            raise Exception(f"Ivalid padding_mode")
        
        #Размеры ядра
        if type(kernel_size) == tuple:
            flter = torch.rand(out_channels, in_channels//groups, kernel_size[0],kernel_size[1],kernel_size[2])
        elif type(kernel_size) == int:
            flter = torch.rand(out_channels, in_channels//groups, kernel_size,kernel_size,kernel_size)
        else:
            raise Exception(f"Ivalid kernel_size type")
            
        #"Обход" ядром
        res = []
        for chnl in range(out_channels):
            feature_map = np.array([])
            for i in range(0, tens.shape[1] - ((flter.shape[2]- 1) * dilation + 1) + 1, stride):
                for j in range(0, tens.shape[2] - ((flter.shape[3]- 1) * dilation + 1) + 1, stride):
                    for k in range(0, tens.shape[3] - ((flter.shape[4]- 1) * dilation + 1) + 1, stride):
                        total = 0
                        for f in range(in_channels // groups):
                            if groups > 1:
                                cur = tens[chnl * (in_channels // groups) + f]\
                                [i:i + (flter.shape[2] - 1) * dilation + 1 : dilation,\
                                 j:j + + (flter.shape[3] - 1) * dilation + 1 : dilation,\
                                 k:k + + (flter.shape[4] - 1) * dilation + 1 : dilation]
                            else:
                                cur = tens[f]\
                                [i:i + (flter.shape[2] - 1) * dilation + 1 : dilation,\
                                 j:j + + (flter.shape[3] - 1) * dilation + 1 : dilation,
                                 k:k + + (flter.shape[4] - 1) * dilation + 1 : dilation]
                            total += (cur * flter[chnl][f]).sum()
                        feature_map = np.append(feature_map, float(total + bias_value[chnl]))
            res.append(feature_map.reshape((tens.shape[1] - ((flter.shape[2] - 1) * dilation + 1)) // stride + 1,\
                          (tens.shape[2] - ((flter.shape[3] - 1) * dilation + 1)) // stride + 1,\
                          (tens.shape[3] - ((flter.shape[4] - 1) * dilation + 1)) // stride + 1))
        return np.array(res),np.array(flter), np.array(bias_value)
    return wrapper

Writing customConv3d.py


In [11]:
%%file test_scenario2.py
from customConv3d import customConv3d
import numpy as np
import torch

test_data_1 = torch.rand(3,28,28,28)
test_data_2 = torch.rand(4,5,6,7)
test_data_3 = torch.rand(1,1,1,1)


def test1():
    customConv = customConv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0\
    , dilation=2, groups=1, bias=True, padding_mode="zeros")
    result, flter, bias_value = customConv(test_data_1)
    torchConv = torch.nn.Conv3d(in_channels=3,out_channels=1, kernel_size=3, stride=1, padding=0\
    , dilation=2, groups=1, bias=True, padding_mode="zeros")
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_1).data),2))
    assert customResult == torchResult

def test2():
    customConv = customConv3d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0\
    , dilation=1, groups=2, bias=True, padding_mode="zeros")
    result, flter, bias_value = customConv(test_data_2)
    torchConv = torch.nn.Conv3d(in_channels=4,out_channels=2, kernel_size=3, stride=1, padding=0\
    , dilation=1, groups=2, bias=True, padding_mode="zeros")
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_2).data),2))
    assert customResult == torchResult
    
def test3():
    customConv = customConv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0\
    , dilation=1, groups=1, bias=True, padding_mode="zeros")
    result, flter, bias_value = customConv(test_data_3)
    torchConv = torch.nn.Conv3d(in_channels=1,out_channels=1, kernel_size=1, stride=1, padding=0\
    , dilation=1, groups=1, bias=True, padding_mode="zeros")
    torchConv.weight.data = torch.tensor(flter)
    torchConv.bias.data = torch.tensor(bias_value)
    customResult = str(np.round(result,2))
    torchResult = str(np.round(np.array(torchConv(test_data_3).data),2))
    assert customResult == torchResult
    
#test1()
#test2()
test3()

Writing test_scenario2.py


In [9]:
from customConv3d import customConv3d
import numpy as np
import torch

test_example = torch.rand(8, 5, 6, 4)

customConv = customConv3d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0\
    , dilation=1, groups=4, bias=True, padding_mode="zeros")
result, flter, bias_value = customConv(test_example)
torchConv = torch.nn.Conv3d(in_channels=8,out_channels=4, kernel_size=3, stride=1, padding=0\
, dilation=1, groups=4, bias=True, padding_mode="zeros")
torchConv.weight.data = torch.tensor(flter)
torchConv.bias.data = torch.tensor(bias_value)

print(result)
print("-------------------------------------------------")
print(np.array(torchConv(test_example).data))

[[[[13.49183846 14.5786171 ]
   [12.56207466 13.0460701 ]
   [13.11169624 13.96009064]
   [13.38720322 14.149189  ]]

  [[12.02116203 12.54479408]
   [12.36138344 12.84284782]
   [13.94972801 14.78197098]
   [13.65278053 12.99024391]]

  [[14.14354134 12.64593124]
   [14.80772018 14.21575928]
   [14.84270859 14.49837017]
   [13.29567909 14.78912354]]]


 [[[13.69804096 12.98000431]
   [13.1160078  13.59095287]
   [13.41521263 13.57130909]
   [13.08943272 15.03600407]]

  [[13.23050404 11.36401272]
   [14.42021656 11.90397549]
   [13.162467   11.8974638 ]
   [13.3615427  13.21050262]]

  [[12.08175564 12.51652241]
   [13.19131565 12.49297905]
   [13.9109726  12.23507118]
   [15.83442783 15.0255022 ]]]


 [[[15.78959751 16.14259911]
   [16.08869743 14.46003723]
   [14.86250019 14.3819418 ]
   [15.42198086 13.29510593]]

  [[15.17127419 14.2552681 ]
   [14.73182869 14.78736401]
   [13.67749023 13.00561428]
   [14.94557095 15.41606617]]

  [[16.94667435 16.85045433]
   [17.11236572 16.6405